In [101]:
import extract
import pandas as pd
import os
import geopandas as gpd

In [102]:
import imp
imp.reload(extract)

<module 'extract' from 'd:\\Users\\User\\Downloads\\RA Work\\acs\\extract.py'>

In [103]:
# open state_fips.csv
state_fips = pd.read_csv('state_fips.csv')
state_fips.head()

,state,code,abbreviation
0,Alabama,1,AL
1,Alaska,2,AK
2,Arizona,4,AZ
3,Arkansas,5,AR
4,California,6,CA


In [117]:
# for every folder in raw_tract_data
for folder in os.listdir('raw_tract_data'):
    # split folder into list by underscore
    folder_tokens = folder.split('_')
    
    # get state column from state_fips where code is third item in list as int
    state = state_fips[state_fips['code'] == int(folder_tokens[2])]['state'].values[0].lower()

    # if last item in folder is 'tract' year is 2000, otherwise it is 20 + last two digits
    year = '2020' if folder_tokens[-1] == 'tract' else '20' + folder_tokens[-1][-2:]

    # if tract file already exists, skip
    if os.path.exists(f'tract_polygons/{state}_tracts_{year}.csv'):
        continue

    # get the filepath of the shp file in the folder with the same name
    shp_file = os.path.join('raw_tract_data', folder, folder + '.shp')

    tract_geos = extract.read_tracts(shp_file)
    
    # save as acs\tract_polygons\berkeley_tracts_2010.csv
    tract_geos.to_csv(f'tract_polygons/{state}_tracts_{year}.csv', index=False)

In [46]:
# for alaska upper (weird KML)
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
districts = gpd.read_file('raw_geo_data\\alaska_upper_2022.kml', driver='KML')
# rename Name to district
districts = districts.rename(columns={'Name': 'district'})
# only keep district and geometry
districts = districts[['district', 'geometry']]
# only keep the part of the district field in the form 0200A
districts['district'] = districts['district'].str.extract(r'(\d{4}[A-Z])')
# only keep the last letter
districts['district'] = districts['district'].str[-1]
# save to geo_polygons folder as alaska_upper_districts_2022.csv
# districts.to_csv('geo_polygons/alaska_upper_districts_2022.csv', index=False)

In [125]:
# for SHP
# read in shp
districts = gpd.read_file(r'raw_geo_data\Minneapolis_Park_Commissioner_Districts_2022\Minneapolis_Park_Commissioner_Districts_2022.shp')
districts

,OBJECTID,MPLSPARK,Shape__Are,Shape__Len,geometry
0,1,1,3.284170e+08,98390.029893,"POLYGON ((-93.22693 45.03567, -93.22693 45.033..."
1,2,2,2.836905e+08,81825.208696,"POLYGON ((-93.31950 45.05125, -93.31823 45.051..."
2,3,3,1.594340e+08,69333.502519,"POLYGON ((-93.22573 44.96616, -93.22286 44.963..."
3,4,4,1.928964e+08,97013.396958,"POLYGON ((-93.26800 44.98965, -93.26765 44.989..."
4,5,5,3.401534e+08,104630.772827,"POLYGON ((-93.21003 44.95469, -93.20776 44.953..."
5,6,6,2.987371e+08,88954.577536,"POLYGON ((-93.32251 44.96048, -93.32252 44.960..."


In [126]:
# only keep district and geometry
district_name = 'MPLSPARK'
year=2022
desc='minneapolis_park_commissioner'
districts = districts[[district_name, 'geometry']]
# rename to district and convert to int
districts = districts.rename(columns={district_name: 'district'})
districts['district'] = districts['district'].astype(int)

In [124]:
# save to geo_polygons folder
districts.to_csv(f'geo_polygons/{desc}_{year}.csv', index=False)